# Homework 1

## Data gathering and cleaning for import into D3 JS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
olympics_dat = pd.read_csv('./exercise2-olympics.csv')
print(olympics_dat.shape)
olympics_dat.head()

In [ ]:
# Remove columns not relevant to our visualization goals
olympics_dat.drop(['Discipline', 'Athlete','City','Country'], axis=1, inplace=True)

# De-dupe Olympics, for country-level placement:
olympics_dd = olympics_dat.copy()
olympics_dd = olympics_dd.drop_duplicates()

# Create numeric column for gold winners:
olympics_dd['gold'] = np.where(olympics_dd['Medal']=='Gold', 1, 0)

In [ ]:
# New dataframe shape
print(olympics_dd.shape)
olympics_dd.head()

In [ ]:
# Create statistics dataframes
country_gold = olympics_dd.groupby(['Year','Country Code'])['gold'].sum().reset_index(name='gold_count')
print(country_gold.shape,": gold size")
country_medal = olympics_dd.groupby(['Year','Country Code']).size().reset_index(name='medal_count').sort_values(['Year','medal_count'],ascending=[True, False])
print(country_medal.shape,": total size")

In [ ]:
# Join them:
olympics_fin = pd.merge(country_medal,country_gold,how='left', left_on=['Year','Country Code'], right_on = ['Year','Country Code'])
print(olympics_fin.shape)
olympics_fin.head()

In [ ]:
# Create one last column, then export results:
olympics_fin['fp_ratio'] = round(olympics_fin['gold_count']/olympics_fin['medal_count'],2)
olympics_fin.head(20)

In [ ]:
# Only keep the top 5, by medal count, each olympics:
olympics_fin_5 = olympics_fin.iloc[olympics_fin.groupby(['Year'])['medal_count'].nlargest(5).index.levels[1]]
olympics_fin_5.head(10)

In [ ]:
# Show final df size:
print(olympics_fin_5.size)

# Check if it is the right amount of data:
len(olympics_fin_5.Year.unique())*5*5==olympics_fin_5.size

In [ ]:
# Gather data for countries, who are the biggest winners:
country_five = olympics_fin.groupby(['Country Code'])['medal_count'].sum().reset_index(name='total_medals').sort_values(['total_medals'],ascending=[False]).head(5)
top_5_list = country_five['Country Code'].tolist()
top_5_list

In [ ]:
# Create filtered data-set based on original:
olympics_fin_top_5 = olympics_fin[olympics_fin['Country Code'].isin(top_5_list)]
print(olympics_fin_top_5.shape)
olympics_fin_top_5.head()

In [ ]:
# Export data:
olympics_fin_5.to_csv('olympics_5.csv',index=False)
olympics_fin_top_5.to_csv('olympics_top_5.csv',index=False)
olympics_fin.to_csv('olympics.csv',index=False)